In [ ]:
def solve_cvrp():

    #uncomment this line of code to install Gurobi if not already installed
    # %pip install gurobipy

    # import the libraries and modules
    import gurobipy as gp
    from gurobipy import GRB
    import json
    import matplotlib.pyplot as plt
    import numpy as np


    with open("cvrp_problem_data.json", "r") as f:
        data = json.load(f)

    # distance imports the distance matrix, a nested list
    distance = data["distance_matrix"]

    # demand imports the customer demand, which is a list
    demand = data["demands"]

    # num_nodes is the total number of nodes
    num_nodes = data["nodes"]["total"]
    depot = data["nodes"]["depot"]

    # customers refer to the delivery locations
    customers = data["nodes"]["delivery_locations"]

    # m is the total number of identical vehicles
    m = data["vehicles"]["count"]

    # Q is the vehicle capacity
    Q = data["vehicles"]["capacity_per_vehicle"]

    N = range(num_nodes)
    C = customers

    # the building of the model starts here
    model = gp.Model("CVRP")

    # decision variables

    # x_ij is a binary variable that indicates the movement of a vehicle from node i to node j
    x_ij = model.addVars([(i, j) for i in N for j in N if i != j], vtype=GRB.BINARY,
                         name="x_ij")

    # f_ij is the amount of load carried on arc (i,j)
    f_ij = model.addVars([(i, j) for i in N for j in N if i != j], lb=0, vtype=GRB.CONTINUOUS,
                         name="f_ij")   

     # objective function

    # the objective function minimizes the total distance traveled by all vehicles
    model.setObjective(gp.quicksum(distance[i][j] * x_ij[i, j] for i, j in x_ij),GRB.MINIMIZE)  

    # constraints

    #  ensures each customer is visited exactly once
    for i in C:
        model.addConstr(gp.quicksum(x_ij[i, j] for j in N if j != i) == 1)
        model.addConstr(gp.quicksum(x_ij[j, i] for j in N if j != i) == 1)

    # the depot degree constraints ensure exactly m vehicles leave and return to the depot
    model.addConstr(gp.quicksum(x_ij[depot, j] for j in C) == m)
    model.addConstr(gp.quicksum(x_ij[i, depot] for i in C) == m)

    # the flow conservation constraint represents the remaining vehicle load
    for i in C:
        model.addConstr(gp.quicksum(f_ij[j, i] for j in N if j != i)-
                        gp.quicksum(f_ij[i, j] for j in N if j != i)
                        == demand[i])

    # the depot flow constraint ensures the total load leaving the depot equals total customer demand
    model.addConstr(gp.quicksum(f_ij[depot, j] for j in C) == sum(demand[i] for i in C))

    # guarantees flow can occur only on used arcs and cannot exceed vehicle capacity
    for i, j in x_ij:
        model.addConstr(f_ij[i, j] <= Q * x_ij[i, j])   


    # this solves the problem
    model.optimize()   


    # the objective which is the optimal total distance is extracted here
    if model.status != GRB.OPTIMAL:
        print("No optimal solution found.")
        exit()

    print(f"\nOptimal total distance: {model.objVal:.2f}")

    # this block builds the adjacency list
    successor = {}
    for i, j in x_ij:
        if x_ij[i, j].X > 0.5:
            successor.setdefault(i, []).append(j)

    # route extraction takes place here
    routes = []
    loads = []

    for i in range(m):
        route = [depot]
        load = 0
        current = depot

        while True:
            next_node = successor[current].pop()
            route.append(next_node)

            if next_node != depot:
                load += demand[next_node]

            current = next_node
            if current == depot:
                break

        routes.append(route)
        loads.append(load)


    # this block outputs the results, which includes the path, load, and distance of each vehicle

    for k, route in enumerate(routes):
        route_distance = 0
        for i in range(len(route) - 1):
            route_distance += distance[route[i]][route[i + 1]]

        print(f"\nVehicle {k + 1} route:")
        print(" -> ".join(map(str, route)))
        print(f"Load: {loads[k]} / {Q}")
        print(f"Distance: {route_distance:.2f}")    


    # visualizing the routes is performed here
    num_routes = len(routes)
    radius = 10  # radius of circle

    x = np.zeros(num_nodes)
    y = np.zeros(num_nodes)

    # positioning of the depot at the center
    x[depot] = 0
    y[depot] = 0

    # this computes the angular span per route
    angle_per_route = 2 * np.pi / num_routes

    for k, route in enumerate(routes):
        route_len = len(route) - 1  # excludes depot at the end
        start_angle = k * angle_per_route
        end_angle = (k + 1) * angle_per_route
        angles = np.linspace(start_angle, end_angle, route_len, endpoint=False)

        # this assigns coordinates for nodes in route (skips depot at the end)
        for ind, node in enumerate(route[:-1]):
            if node == depot:
                continue
            x[node] = radius * np.cos(angles[ind])
            y[node] = radius * np.sin(angles[ind])


    # the plot starts here

    plt.figure(figsize=(10, 8))
    colors = ["tab:blue", "tab:red"]
    show_node_labels = True

    for k, route in enumerate(routes):
        route_x = [x[i] if i != depot else 0 for i in route]
        route_y = [y[i] if i != depot else 0 for i in route]

        # drawing of the route lines is done here
        plt.plot(route_x, route_y, marker="o", color=colors[k % len(colors)], linewidth=2,
                 label=f"Vehicle {k + 1}")

        # this adds arrows only on the first and last edge
        plt.arrow(route_x[0], route_y[0], route_x[1] - route_x[0], route_y[1] - route_y[0],
                  shape='full', lw=0, length_includes_head=True, head_width=0.3,
                  color=colors[k % len(colors)])
        plt.arrow(route_x[-2], route_y[-2], route_x[-1] - route_x[-2],
                  route_y[-1] - route_y[-2], shape='full', lw=0, length_includes_head=True,
                  head_width=0.3, color=colors[k % len(colors)])

        # this block focuses on the centroid for load & distance annotation
        centroid_x = np.mean(route_x)
        centroid_y = np.mean(route_y)
        route_distance = sum(distance[route[i]][route[i + 1]] for i in range(len(route) - 1))
        plt.text(centroid_x, centroid_y + 0.5,  # slightly above centroid
                 f"Load: {loads[k]} / {Q}\nDistance: {route_distance:.0f}", fontsize=10,
                 fontweight="bold", color=colors[k % len(colors)], ha='center')

    # this visualizes the depot
    plt.scatter(0, 0, color="black", s=200, marker="s", label="Depot", zorder=5)

    # here are the node labels
    if show_node_labels:
        for i in range(num_nodes):
            plt.text(x[i] + 0.3, y[i] + 0.3, str(i), fontsize=9)

    plt.title("Routes Visualization")
    plt.axis("off")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
solve_cvrp()